In [0]:
from pyspark.sql.functions import current_timestamp, lit

# --- Configuração dos Caminhos ---

# 1. Definição da conta de armazenamento
storage_account_name = "stacdatatrabalhoed01"

# Caminho para os dados brutos para upload
landing_zone_path = f"abfss://landingzone@{storage_account_name}.dfs.core.windows.net"

bronze_path = f"abfss://bronze@{storage_account_name}.dfs.core.windows.net"

In [0]:
# Lista de todas as tabelas
tables = [
    "agendamento", "consulta", "consulta_procedimento", "endereco",
    "log_pagamento", "odontologista", "paciente", "pagamento",
    "procedimento", "tipo_pagamento"
]

# --- Loop de Ingestão ---
for table_name in tables:
    print(f"Processando tabela: {table_name}")

    # 1. Ler o arquivo CSV da Landing Zone
    df_raw = spark.read.format("csv") \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .load(f"{landing_zone_path}/{table_name}.csv")

    # 2. Adicionar metadados de ingestão (boa prática)
    df_with_metadata = df_raw.withColumn("data_ingestao_bronze", current_timestamp()) \
                               .withColumn("fonte_dados", lit(f"{table_name}.csv"))

    # 3. Escrever como tabela Delta na camada Bronze
    # O modo "overwrite" substitui os dados a cada execução.
    df_with_metadata.write.format("delta") \
        .mode("overwrite") \
        .save(f"{bronze_path}/{table_name}")

    # (Opcional) Criação de uma tabela no metastore do Databricks para facilitar consultas SQL
    spark.sql(f"CREATE TABLE IF NOT EXISTS bronze_{table_name} USING DELTA LOCATION '{bronze_path}/{table_name}'")

print("Processo de ingestão para a camada Bronze concluído.")

Processando tabela: agendamento
Processando tabela: consulta
Processando tabela: consulta_procedimento
Processando tabela: endereco
Processando tabela: log_pagamento
Processando tabela: odontologista
Processando tabela: paciente
Processando tabela: pagamento
Processando tabela: procedimento
Processando tabela: tipo_pagamento
Processo de ingestão para a camada Bronze concluído.
